# Notebook Edge detection (lectures 4 & 5)

*Notes explicatives et pratiques des notions abordées.*   

## Linear Systems
*interprétation personnelle*\
Dans le cadre de l'utilisation de filtres, on peut qualifié de système une fonction permettant de passer d'une représentation fonctionnelle de l'image à une autre. En effet, si l'on se réfère au chapitre précédant, nous pouvons imaginer une image comme une fonction discrète 2D (définie pour des valeurs entières représentant les indices de ligne et de colonne au sein de l'image).
Nous pouvons flouter l'image, dans ce cas là l'image correspond à une autre fonction discrète.

La transformation (et même plus précisément le filtre) permettant de passer de la première représentation de l'image (la première fonction) à la seconde représentation (la seconde fonction) est qualifié de système. 

Si le système avec lequel nous travaillons est qualifié de linéaire, alors si nous appliquons 2 transformations appartenant au système, sur notre image, alors nous devons avoir:
$$
S[\alpha f_i[n, m] + \beta f_j[n, m]] = \alpha S[f_i[n, m]] + \beta S[f_j[n, m]]
$$

Cela signifie que la résultante des 2 transformations est identiques à somme (pondérées) de chacune des résultantes prises séparément.

In [12]:
import numpy as np
import cv2

origin_arr = np.identity(10) + np.rot90(np.identity(10))

In [21]:
output1 = origin_arr / np.sum(origin_arr)
output2 =  2 * origin_arr - 2

In [22]:
0.5 * output1 + 0.5 * output2

array([[ 0.025, -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,
        -1.   ,  0.025],
       [-1.   ,  0.025, -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,
         0.025, -1.   ],
       [-1.   , -1.   ,  0.025, -1.   , -1.   , -1.   , -1.   ,  0.025,
        -1.   , -1.   ],
       [-1.   , -1.   , -1.   ,  0.025, -1.   , -1.   ,  0.025, -1.   ,
        -1.   , -1.   ],
       [-1.   , -1.   , -1.   , -1.   ,  0.025,  0.025, -1.   , -1.   ,
        -1.   , -1.   ],
       [-1.   , -1.   , -1.   , -1.   ,  0.025,  0.025, -1.   , -1.   ,
        -1.   , -1.   ],
       [-1.   , -1.   , -1.   ,  0.025, -1.   , -1.   ,  0.025, -1.   ,
        -1.   , -1.   ],
       [-1.   , -1.   ,  0.025, -1.   , -1.   , -1.   , -1.   ,  0.025,
        -1.   , -1.   ],
       [-1.   ,  0.025, -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,
         0.025, -1.   ],
       [ 0.025, -1.   , -1.   , -1.   , -1.   , -1.   , -1.   , -1.   ,
        -1.   ,  0.025]])

## 3. Edge Detection for Computer Vision
Le but de la détection des bords est d'identifier les variations soudaines (les discontinuités) au sein d'une image.
De manière intuitive, la majorité de l'information sémantique peut être associée aux bords.

Les bords nous aident à extraire des informations dans ce que nous voyons, à reconnaître des objets, des formes géométriques et appréhender le point de vue avec lequel nous observons une scène.
Les bords se "forment"/définissent par la discontinuité d'une surface, la variation brutale de la profondeur, des couleurs ou de la luminosité.

### 3.1 Les dérivées discrètres en 1D:
> **Définition 1:**\
> La dérivée *"vers l'arrière"*
> $$
> \frac{df}{dx} = f(x) - f(x-1) = f'(x)
> $$
>

> **Définition 2:**\
> La dérivée *"vers l'avant"*
> $$
> \frac{df}{dx} = f(x) - f(x+1) = f'(x)
> $$
>


> **Définition 3:**\
> La dérivée *"centrée"*
> $$
> \frac{df}{dx} = f(x+1) - f(x-1) = f'(x)
> $$
>


In [32]:
def apply_derivative(arr:np.array, derivative: str):
    """ Depending of the derivative function, the first, last or first and last elements of the array must be skipped

    Args:
        arr (np.array): 1D array to differenciate
        derivative (str): name of the definition of the derivative applied ("backward", "forward" or "centred")
    """
    output = []
    if derivative == "backward":
        output.append(None) # None pour signifier que l'on skip en theorie la premiere dérivée
        for i in range(1, len(arr)):
            diff = arr[...] - arr[...] # f(x) - f(x-1)
            output.append(diff)
    elif derivative == "forward":
        for i in range(len(arr) - 1):
            diff = arr[...] - arr[...]  # f(x) - f(x+1)
            output.append(diff)
        output.append(None) # None pour signifier que l'on skip en theorie la premiere dérivée
    elif derivative == "centred":
        output.append(None) # None pour signifier que l'on skip en theorie la premiere dérivée
        for i in range(1, len(arr) - 1):
            diff = arr[...] - arr[...]  # f(x+1) - f(x-1)
            output.append(diff)
        output.append(None)  # None pour signifier que l'on skip en theorie la derniere dérivée
    return np.array(output)

In [30]:
arr = np.random.randint(0, 10, size=15)

diff_backward = apply_derivative(arr, "backward")
diff_forward = apply_derivative(arr, "forward")
diff_centred = apply_derivative(arr, "centred")

In [31]:
print("input array  :", arr.reshape(1, -1))
print("backward diff:", diff_backward.reshape(1, -1))
print("forwward diff:", diff_forward.reshape(1, -1))
print("centred diff :", diff_centred.reshape(1, -1))

input array  : [[3 3 0 9 7 2 7 0 2 1 1 8 3 0 4]]
backward diff: [[None 0 -3 9 -2 -5 5 -7 2 -1 0 7 -5 -3 4]]
forwward diff: [[0 3 -9 2 5 -5 7 -2 1 0 -7 5 3 -4 None]]
centred diff : [[None -3 6 7 -7 0 -2 -5 1 -1 7 2 -8 1 None]]


Voici un exemple de ce que j'ai pu obtenir:
```python
input array  : [[3 3 0 9 7 2 7 0 2 1 1 8 3 0 4]]
backward diff: [[None 0 -3 9 -2 -5 5 -7 2 -1 0 7 -5 -3 4]]
forwward diff: [[0 3 -9 2 5 -5 7 -2 1 0 -7 5 3 -4 None]]
centred diff : [[None -3 6 7 -7 0 -2 -5 1 -1 7 2 -8 1 None]]
```

### 3.2 Discrete derivative in 2D:
> **Définition: Vecteur gradient**
> $$
> \nabla f(x, y) = \begin{bmatrix} \frac{\partial f}{\partial x} \\ \frac{\partial f}{\partial y} \end{bmatrix} 
> $$

> **Définition: Amplitude du vecteur gradient**
> $$
> \left\lVert \nabla f(x, y) \right\rVert = \sqrt{\frac{\partial f}{\partial x}^2 + \frac{\partial f}{\partial y}^2}
> $$

> **Définition: Direction du vecteur gradient**
> $$
> \theta = arctan\left(\frac{\frac{\partial f}{\partial y}}{\frac{\partial f}{\partial x}}\right)
> $$

### 3.3 Example (and approximation)
Le gradient d'une matrice aux coordonnées $(i,j)$ peut être approximé en utilisant les valeurs des pixels autour du pixel d'intérêt (le pixel central de coordonnée $(i,j)$) via le filtre:
$$
\frac{1}{3}
\begin{bmatrix}
-1 & 0 & 1 \\
-1 & 0 & 1 \\
-1 & 0 & 1
\end{bmatrix}
$$

## Application